# Quick Exploration EDT

In [ ]:
#imports
import pandas as pd
import numpy as np
from google.colab import drive
from collections import Counter

In [ ]:
# mount my google drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
file_path_EDT = {
    'train':'/content/drive/MyDrive/DATASCI_266_NLP/EDT_Copy/train.txt',
    'test': '/content/drive/MyDrive/DATASCI_266_NLP/EDT_Copy/dev.txt',
    }

In [ ]:
train_EDT = pd.read_csv(file_path_EDT['train'], sep='\t', header=None)
train_EDT.head()
train_EDT[1].value_counts()

,count
1,
O,2958299
I-NC,20164
I-SR,12113
I-CT,11532
I-A,7648
I-RD,7577
I-DI,6328
I-DC,2630
I-GC,2178


In [ ]:
test_EDT[1].value_counts()

,count
1,
O,744436
I-NC,4895
I-CT,3358
I-SR,2992
I-A,1844
I-RD,1735
I-DI,1696
I-DC,676
I-GC,665


## Combining into sentences not words
- using the last label doesn't seem to make sense it creates almost all unknowns

In [ ]:
with open(file_path_EDT['train'], 'r') as file:
    lines = file.readlines()

# Initialize variables
sentences = []
current_sentence = []
current_labels = []

# Function to extract unique labels
def get_unique_label(labels):
    unique_labels = list(set(labels))  # return unique labels

    if len(unique_labels) > 1 and 'O' in unique_labels:
        unique_labels.remove('O')
    return unique_labels

# alternative funciton to get max value label
def get_summary_label(labels):
  # get unique values
  unique_labels = list(set(labels))

  # if len greater than 1
  if len(unique_labels) > 1 and 'O' in unique_labels:
    unique_labels.remove('O')
    if len(unique_labels) > 1:
      label_counts = Counter(labels)  # Count occurrences of each label
      max_label = label_counts.most_common(1)[0][0]
      return max_label[0]
    else:
      return unique_labels[0]
  else:
      return unique_labels[0]


# Function to process a file and convert it to a DataFrame
def process_file(file_path):
    with open(file_path, 'r') as file:
        lines = file.readlines()

    sentences = []
    current_sentence = []
    current_labels = []

    # Iterate over each line in the file
    for line in lines:
        line = line.strip()  # Remove leading/trailing whitespace/newlines
        if line:  # If the line is not empty, process it
            word, label = line.split('\t')
            current_sentence.append(word)  # Store the word
            current_labels.append(label)  # Store the label
        else:  # Empty line indicates end of a sentence
            if current_sentence:
                sentence = ' '.join(current_sentence)
                unique_label = get_unique_label(current_labels)
                summary_label = get_summary_label(current_labels)
                sentences.append({'0': sentence, '1': summary_label, '2':unique_label})
                current_sentence = []  # Reset current sentence
                current_labels = []  # Reset current labels

    # If there's a sentence left after the last line, add it
    if current_sentence:
        sentence = ' '.join(current_sentence)
        summary_label = get_summary_label(current_labels)
        sentences.append({'0': sentence, '1': unique_label})

    return pd.DataFrame(sentences)

In [ ]:
df_train = process_file(file_path_EDT['train'])
df_test = process_file(file_path_EDT['test'])

In [ ]:
df_train.head()

,0,1,2
0,JTI report warns of a 'Gathering Storm' in the...,O,[O]
1,Harmonic Announces IBC 2020 Edition of Live Co...,O,[O]
2,"AF Group Again Named To ""Best Places to Work i...",O,[O]
3,SMSbiotech Engages Exemplar Capital for Strate...,O,[O]
4,Statement From Louis Audet With Regard to the ...,O,[O]


In [ ]:
df_train['2'].value_counts()

,count
2,
[O],5963
[I-NC],413
[I-SR],268
[I-CT],251
[I-A],183
[I-RD],170
[I-DI],139
[I-DC],82
[I-GC],78


In [ ]:
df_train['1'].value_counts()

,count
1,
O,6051
I-NC,413
I-SR,268
I-CT,251
I-A,183
I-RD,170
I-DI,139
I-DC,82
I-GC,78


In [ ]:
df_test.head()

,0,1
0,"Frisch's Restaurants , Inc . Clarifies Payment...",[I-SD]
1,Ethan Allen Declares Special Dividend and Regu...,"[I-RD, I-SD]"
2,W . R . Berkley Corporation Declares Special D...,"[I-RD, I-SD]"
3,CoreLogic to Acquire Symbility Solutions TORON...,[I-A]
4,"Expedia , Inc . Announces Special Dividend BEL...",[I-SD]


In [ ]:
df_test['1'].value_counts()

,count
1,
O,1507
I-NC,103
I-SR,68
I-CT,62
I-RD,48
I-A,46
I-DI,39
I-DC,21
I-GC,20


In [ ]:
df_test['2'].value_counts()

,count
2,
[O],1492
[I-NC],103
[I-SR],68
[I-CT],62
[I-RD],48
[I-A],46
[I-DI],39
[I-DC],21
[I-GC],20
